In [37]:
import torch
from torch.utils.data import DataLoader,Dataset
from torchvision import models, transforms
import cv2
import json
import numpy as np

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
class KeyPointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w 
        kps[1::2] *= 224.0 / h 

        return img, kps

In [40]:
train_dataset = KeyPointsDataset(r"C:\Users\aashutosh kumar\OneDrive\Pictures\data\images",r"C:\Users\aashutosh kumar\OneDrive\Pictures\data\data_train.json")
val_dataset = KeyPointsDataset(r"C:\Users\aashutosh kumar\OneDrive\Pictures\data\images",r"C:\Users\aashutosh kumar\OneDrive\Pictures\data\data_train.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [41]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2)

c:\a\envs\torch_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\a\envs\torch_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
model = model.to(device)

In [43]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [44]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14906.279296875
Epoch 0, iter 100, loss: 10794.3349609375
Epoch 0, iter 200, loss: 7149.91650390625
Epoch 0, iter 300, loss: 5043.7724609375
Epoch 0, iter 400, loss: 2961.211181640625
Epoch 0, iter 500, loss: 2009.10107421875
Epoch 0, iter 600, loss: 982.9390869140625
Epoch 0, iter 700, loss: 557.607177734375
Epoch 0, iter 800, loss: 270.8610534667969
Epoch 1, iter 0, loss: 311.3451843261719
Epoch 1, iter 100, loss: 121.4499740600586
Epoch 1, iter 200, loss: 63.66838836669922
Epoch 1, iter 300, loss: 60.92205047607422
Epoch 1, iter 400, loss: 63.991214752197266
Epoch 1, iter 500, loss: 37.45306396484375
Epoch 1, iter 600, loss: 32.99452590942383
Epoch 1, iter 700, loss: 44.997318267822266
Epoch 1, iter 800, loss: 23.03718376159668
Epoch 2, iter 0, loss: 15.127215385437012
Epoch 2, iter 100, loss: 17.711627960205078
Epoch 2, iter 200, loss: 23.396812438964844
Epoch 2, iter 300, loss: 20.0445499420166
Epoch 2, iter 400, loss: 24.645822525024414
Epoch 2, iter 500, l